# Classification using the Inception-ResNet-v2

This notebook is based on the code from Benjamin Wild, with modifications and additions.

First, we load the required dependencies and set the paths to the files.

In [ ]:
# Config
import os
import sys
base_path = os.path.abspath(os.pardir)
sys.path.append(base_path)

print(os.getcwd())

os.chdir(base_path + '/data')
data_path = 'model_data_extended/'
#data_path = '/home/ec2-user/IZW_Detector/cheetah_data/model_data_extended/'
train_data_path = data_path + 'train/'
val_data_path = data_path + 'val/'
all_data_path = '/home/ec2-user/IZW_Detector/cheetah_data/model_data_extended/all'

In [ ]:
%matplotlib inline

import tensorflow as tf
from keras.backend.tensorflow_backend import set_session, get_session
config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.9
config.gpu_options.allow_growth = True
set_session(tf.Session(config=config))

import keras.backend as K
from keras.applications.inception_resnet_v2 import InceptionResNetV2, preprocess_input
from keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from keras.layers import Dense, Dropout, Input, concatenate
from keras.models import Model, load_model
from keras.optimizers import Nadam
from scipy.ndimage.interpolation import rotate
from sklearn.metrics import precision_recall_curve, classification_report, accuracy_score, confusion_matrix
import pandas as pd
import numpy as np
import itertools
import matplotlib.pyplot as plt

In [ ]:
print(sys.path)

# Monkey-patch keras DirectoryIterator to also return filename
import keras
from notebooks.keras_util.util import DirectoryIteratorWithFname
#import keras_preprocessing
#from notebooks.keras_util_v2.util import DirectoryIteratorWithFname
keras.preprocessing.image.DirectoryIterator = DirectoryIteratorWithFname

---
## Data loading

We load the text file containing the labels used in ImageNet. Cheetahs and Leopards are already present there.

In [ ]:
# Load and parse ImageNet class labels

#classes = open(base_path + '/modules/imagenet_classes', 'r').readlines()
classes = open('imagenet_classes', 'r').readlines()

def strip(c):
    key, value = c.split(':')
    key = key.strip()
    key = key.split('{')[-1]
    value = value.split("'")[1].strip()
    return int(key), value

classes = dict([strip(c) for c in classes])

In [ ]:
izw_classes = ('unknown', 'cheetah', 'leopard')

---
We load the metadata (which contains the file paths) generated in the `split_data` notebook.

In [ ]:
metadata = pd.read_hdf(data_path + 'metadata.hdf5')
val_metadata = metadata[metadata.set=='val'].copy()

class_counts = metadata.label.value_counts(normalize=True)

from sklearn.utils import class_weight

cheetah_weight = class_weight.compute_class_weight('balanced', izw_classes, metadata.label)
cheetah_weight = dict(enumerate(cheetah_weight))

class_proportions = np.array([class_counts[label] for label in izw_classes])
class_dict = {index:val for (index, val) in enumerate(class_proportions)}

print(cheetah_weight)
print(class_dict)

metadata.head()

---
The Keras image generators read the training and validation sets and supply a stream of image batches of the classes. (They infer the image classes from the directory names.)  
The training image iterator additionally augments the dataset by flipping images, the validation image iterator does not. 

Additionally there's a preprocessing generator wrapped around the iterator which augments the image input data with time/temperature metadata and additionally rotates images. 

In [ ]:
# Crop camera metainformation from images

def preprocess(data, batch_size, rotate_range=None):
    for x, y, fns in data:
        batch_metadata = []
        for fname in fns:
            fname_splitted = fname.split('_')[1].split('.')
            
            index = fname_splitted[0]
            #rest = '_'.join(fname_splitted[1:]).split('.jpeg')[0]
            f_metadata = metadata.loc[int(index)]
            
            batch_metadata.append((
                f_metadata.ambient_temp,
                f_metadata.hour))
            # optionally use metadata
        temperatures = np.array(batch_metadata).astype(np.float32)
        x = x[:, 10:-10, 10:-10, :]
        if rotate_range is not None:
            for idx in range(x.shape[0]):
                x[idx] = rotate(x[idx], np.random.random() * rotate_range * 2 - rotate_range, 
                                order=1, mode='reflect', reshape=False).astype(np.int64)
        yield [preprocess_input(x), temperatures], y
        #yield [2*((x/255)-0.5), temperatures], y

In [ ]:
# configure the batch size
batch_size = 32
val_batch_size = 32

all_size, train_size, val_size = sum(metadata.set.isin(['train', 'val'])), sum(metadata.set=='train'), sum(metadata.set=='val')
print(all_size, train_size, val_size)

# Augment train data with horizontal flips, scale to ImageNet input size

generator = ImageDataGenerator(horizontal_flip=True)
all_generator = ImageDataGenerator(horizontal_flip=True)
val_generator = ImageDataGenerator(horizontal_flip=False)

train_gen = preprocess(generator.flow_from_directory(
    train_data_path, 
    target_size=(299+20, 299+20),
    classes=izw_classes,
    batch_size=batch_size), batch_size, rotate_range=10)

all_gen = preprocess(all_generator.flow_from_directory(
    all_data_path, 
    target_size=(299+20, 299+20),
    classes=izw_classes,
    batch_size=batch_size), batch_size, rotate_range=10)

val_gen = preprocess(val_generator.flow_from_directory(
    val_data_path, 
    target_size=(299+20, 299+20),
    classes=izw_classes,
    batch_size=val_batch_size), val_batch_size)

---
Because training the whole model with 32 image batches doesn't fit our 4GB GPU, we specifiy additional generators for batches of size 4.

The top layers can be trained with 32 size batches, but the finetuning only works on 4 image batches. Adjust if trained on a better GPU.

---
# Test pretrained model

To get a classification baseline of the neural net without finetuning, we just try to classify with some pictures.  
It generally works somewhat, but the NN has a harder job than necessary because the best estimates sometimes are labels we are not interested in.

In [ ]:
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

    thresh = cm.max() / 1.001
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        s = '{:.3f}'.format(cm[i, j]) if normalize else cm[i,j]
        plt.text(j, i, s,
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    
def label_confusion_matrix(val_labels, val_preds):
    cnf_matrix = confusion_matrix(val_labels, val_preds)
    plt.figure(figsize=(6, 6))
    plot_confusion_matrix(cnf_matrix, classes=izw_classes,
                          normalize=False, title='Confusion matrix')

In [ ]:
# Load pretrained model
#
# http://arxiv.org/abs/1602.07261
#
# Inception-v4, Inception-ResNet and the Impact of Residual Connections
# on Learning
#
# Christian Szegedy, Sergey Ioffe, Vincent Vanhoucke, Alex Alemi

model = InceptionResNetV2()

In [ ]:
val_labels = []
val_preds = []

for idx, (batch, labels) in enumerate(val_gen):
    pred = model.predict(batch[0])
    for i in range(pred.shape[0]):    
        class_pred = classes[pred[i,:].argmax()]
        class_pred = np.where([x in class_pred for x in izw_classes])
        class_pred = 0 if class_pred[0].size==0 else class_pred[0][0]
        
        val_preds.append(class_pred)
        val_labels.append(labels[i].argmax())
    
    if idx == (val_size // batch_size):
        break

In [ ]:
print('Accuracy: {:.3f}%\n'.format(accuracy_score(val_labels, val_preds) * 100))
print(classification_report(val_labels, val_preds, target_names=izw_classes))

label_confusion_matrix(val_labels, val_preds)

---
# Model definition and finetuning

First we define the model: Based on the InceptionResnetV2, we remove its last fully connected layer.  
A new final layer is added that takes the previous hidden layer output and the metadata information. Dropout is used for regularization. 

We only have a 3-class softmax output for the Cheetah, Leopard and Unknown classes.

In [ ]:
from keras.callbacks import ModelCheckpoint

path_prefix = "/home/ec2-user/cheetah_data/models/"

# define callbacks to save the model
filepath= path_prefix + "weights-improvement-{epoch:02d}-{val_acc:.2f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

In [ ]:
# Use pretrained model, but skip last (classifier) layer and replace it with a new layer for the three IZW classes
# Fix pretrained layers, only learn last weights for last layer
# Also use metadata as additional input

metadata_input = Input(shape=(2, ))

base_model = InceptionResNetV2(include_top=False, pooling='avg')
h = concatenate([base_model.output, metadata_input])
h = Dropout(.2, name='Dropout')(h)
outputs = Dense(3, activation='softmax')(h)

for layer in base_model.layers:
    layer.trainable = False
    
model = Model(base_model.inputs + [metadata_input], outputs)

In [ ]:
model.load_weights(path_prefix + 'weights-improvement-03-0.98.hdf5')
optim = Nadam(0.001)
model.compile(loss='categorical_crossentropy', optimizer=optim, metrics=['accuracy'])

---
Training the last layer is straightforward, but finetuning the whole model afterwards can be difficult since it requires a GPU with much memory.

Here we train the model, evaluate the results and store the weights and the model on disk.

The cells are currently deactivated unless we want to retrain.

In [ ]:
model.save(path_prefix + "base_model.hdf5")

In [ ]:
optim = Nadam(0.001)
model.compile(loss='categorical_crossentropy', optimizer=optim, metrics=['accuracy'])
model.fit_generator(train_gen, steps_per_epoch=train_size // batch_size, validation_data=(val_gen), 
                    validation_steps=val_size // val_batch_size, epochs=3, workers=4, use_multiprocessing=True,
                    callbacks=callbacks_list)
                    #callbacks=callbacks_list, class_weight=cheetah_weight)

In [ ]:
# Now finetune the whole model for five more epochs with reduced learning rate

for layer in base_model.layers:
    layer.trainable = True

optim = Nadam(0.0001)
model.compile(loss='categorical_crossentropy', optimizer=optim, metrics=['accuracy'])
#model.fit_generator(train_gen, steps_per_epoch=train_size // batch_size, validation_data=(val_gen), 
model.fit_generator(all_gen, steps_per_epoch=all_size // batch_size, validation_data=(val_gen), 
        validation_steps=val_size // val_batch_size, epochs=6, workers=4, use_multiprocessing=True,
        callbacks=callbacks_list, initial_epoch=4)

In [ ]:
import time
a = time.time()
print(model.evaluate_generator(val_gen, steps=val_size // val_batch_size, workers=1, use_multiprocessing=True))
print(time.time() - a)

In [ ]:
#model.save(path_prefix + 'model_v3.hdf5')
model.save_weights(filepath.format(epoch=5, val_acc=0.99))

---
# Evaluation

The model now seems to more reliably detect the labels we are interested in (it's easier to decide among three classes than 1000).
We store the class probabilities of the predictions in the validation dataframe copy.

In [ ]:
val_metadata['predict_probs'] = 0
val_metadata['predict_probs'] = val_metadata['predict_probs'].astype(object)

for event_key, group in val_metadata.groupby('event_key_simple'):
    batch_x = np.zeros((len(group), 299+20, 299+20, 3), dtype=K.floatx())
    batch_t = np.zeros((len(group), ), dtype=K.floatx())
    batch_h = np.zeros((len(group), ), dtype=K.floatx())
    group_iterator = enumerate(zip(group.index, group.set_path, group.ambient_temp, group.hour))
    for i, (file_idx, path, temp, hour) in group_iterator:
        file_path = os.path.join(*([val_data_path] + path.split('/')[-2:]))
        img = load_img(file_path, grayscale=False, target_size=(299+20, 299+20))
        x = img_to_array(img)
        x = val_generator.random_transform(x.astype(K.floatx()))
        x = val_generator.standardize(x)
        batch_x[i] = x
        batch_t[i] = temp
        batch_h[i] = hour
    batch_x = batch_x[:, 10:-10, 10:-10, :]
    batch_x = [preprocess_input(batch_x), np.stack((batch_t, batch_h), axis=1)]
    pred = model.predict_on_batch(batch_x)
    
    for i, (row_index, row) in enumerate(group.iterrows()):
        val_metadata.at[row_index, 'predict_probs'] = pred[i,:]

We can do a preliminary analysis on the single images (ignoring the correlated 3-image events) and estimate our new accuracy, which is also much improved compared to the naive model:

In [ ]:
izw_class_arr = np.array(izw_classes)

val_preds = val_metadata['predict_probs'].apply(np.argmax).as_matrix()
val_probs = np.vstack(val_metadata['predict_probs'])
val_labels = val_metadata['label'].apply(lambda x: np.where(x == izw_class_arr)[0][0]).as_matrix()

In [ ]:
print('Accuracy: {:.3f}%\n'.format(accuracy_score(val_labels, val_preds) * 100))
print(classification_report(val_labels, val_preds, target_names=izw_classes))

label_confusion_matrix(val_labels, val_preds)

In [ ]:
cheetah_val_probs = val_probs[:, izw_classes.index('cheetah')]
cheetah_val_labels = val_labels == izw_classes.index('cheetah')

precision, recall, thresholds = precision_recall_curve(cheetah_val_labels, cheetah_val_probs)

In [ ]:
plt.style.use('seaborn-notebook')
plt.figure(figsize=(8, 6))
plt.plot(recall, precision)
plt.plot(recall[precision > .999], precision[precision > .999], c='green')
plt.plot(recall[recall > .999], precision[recall > .999], c='green')
plt.ylabel('Precision')
plt.xlabel('Recall')
plt.title('Precision-Recall curve for cheetahs')
plt.ylim([precision.min()-0.01, precision.max()+0.01])
plt.xlim([-0.05, 1.05])

In [ ]:
precision_max_recall = precision[np.where(recall > .999)[0][-1]]
print('Precision for cheetahs for recall > 99.9%: {:.1f}%'.format(precision_max_recall * 100))

---
# Classification analysis - simple events

Individual images are classified quite well. We now want to explore how series of events can be optimally classified.

We explore three strategies:

- Use the image with the highest probability classification as the label for the 3-image events.
- Sum the class probabilities in the 3-image events and take the highest probabilty. (Amounts to averaging over the images.)
- Label the 3 images separately and majority vote the label, break ties among 3 classes by probability.

First, we explore the prevalence of 3-image events with conflicting predicted labels according the the maximum class probabilities:

In [ ]:
def get_event_labels(data_group):
    # count the number of events where we have different labels
    label_counts = np.array([0,0,0,0])
    # count the number of times any of the labels were correct
    correct_label = np.array([0,0,0,0])
    
    event_lens = np.array([0,0,0,0])

    for index, group in data_group:
        group_label = izw_classes.index(group['label'].iloc[0])
        labels = group['predict_probs'].apply(np.argmax).unique()
        label_count = len(labels)
        label_counts[label_count] += 1
        event_lens[label_count] += len(group)
        correct_label[label_count] += 1 if group_label in labels else 0
        
    return label_counts, correct_label, event_lens

label_counts, correct_label, event_lens = get_event_labels(val_metadata.groupby('event_key_simple'))
print("Observed label counts: {}".format(label_counts))
print("Correct label in set:  {}".format(correct_label))
print("Correct label in set:  {}".format(event_lens))

---
### Unique-label event accuracy
Most of the events only have one label, of which almost all are correct. Of the few events which have 2 or even 3 labels, the correct label is at least among the options (which is of course trivially true for the 3 label cases).

We can't do much about the events where all images are mislabeled, but we can look at the confusion matrix for these. It shows that 2 images are false positives (a harmless case) while the others, except 1 missed Leopard, confuse Cheetahs and Leopards. 

In [ ]:
def unique_label_predictions(data_group, normalize_counts = False):
    group_labels, group_preds = [], []
    for index, group in data_group:
        labels = group['predict_probs'].apply(np.argmax).unique()
        group_label = izw_classes.index(group['label'].iloc[0])
        if len(labels) == 1:     
            factor = len(group) if normalize_counts else 1
            group_preds.extend([labels[0]] * factor)
            group_labels.extend([group_label] * factor)
            
    return group_labels, group_preds

group_labels, group_preds = unique_label_predictions(val_metadata.groupby('event_key_simple'))
print('Accuracy: {:.3f}%\n'.format(accuracy_score(group_labels, group_preds) * 100))
print(classification_report(group_labels, group_preds, target_names=izw_classes, digits=4))

label_confusion_matrix(group_labels, group_preds)

In [ ]:
group_labels, group_preds = unique_label_predictions(val_metadata.groupby('event_key_simple'), normalize_counts=True)
print('Accuracy: {:.3f}%\n'.format(accuracy_score(group_labels, group_preds) * 100))
print(classification_report(group_labels, group_preds, target_names=izw_classes, digits=4))

label_confusion_matrix(group_labels, group_preds)

---
### Conflicting-label event accuracy
We now try the labeling strategies (maximum probability vs. sum probabilities) for the muliply labeled cases.

It seems as if the selection based on the maximum class probability for any of the 3 images gives better results.

In [ ]:
def all_label_predictions(data_group, normalize_counts = False):
    group_labels_all, group_preds_all, group_labels, group_preds1, group_preds2 = [], [], [], [], []
    for index, group in data_group:
        group_probs = np.vstack(group['predict_probs'])
        labels = group['predict_probs'].apply(np.argmax).unique()
        group_label = izw_classes.index(group['label'].iloc[0])
        factor = len(group) if normalize_counts else 1

        if len(labels) > 1:
            group_labels_all.extend([group_label]*factor)
            group_labels.extend([group_label]*factor)
            label1 = np.unravel_index(group_probs.argmax(), group_probs.shape)[1]
            label2 = np.sum(group_probs, axis=0).argmax()
            group_preds_all.extend([label1]*factor)
            group_preds1.extend([label1]*factor)
            group_preds2.extend([label2]*factor)
        else:
            group_labels_all.extend([group_label]*factor)
            group_preds_all.extend([labels[0]]*factor)
            
    return group_labels_all, group_preds_all, group_labels, group_preds1, group_preds2
        
group_labels_all1, group_preds_all1, group_labels, group_preds1, group_preds2 = all_label_predictions(val_metadata.groupby('event_key_simple'), normalize_counts=False)
print('\t=========   Normalized   =========')
print('Accuracy: {:.3f}%\n'.format(accuracy_score(group_labels, group_preds1) * 100))
print(classification_report(group_labels, group_preds1, target_names=izw_classes, digits=6))
label_confusion_matrix(group_labels, group_preds1)

group_labels_all2, group_preds_all2, group_labels, group_preds1, group_preds2 = all_label_predictions(val_metadata.groupby('event_key_simple'), normalize_counts=True)
print('Accuracy: {:.3f}%\n'.format(accuracy_score(group_labels, group_preds2) * 100))
print(classification_report(group_labels, group_preds2, target_names=izw_classes, digits=6))
label_confusion_matrix(group_labels, group_preds1)

---
### Overall event accuracy
The accuracy for the whole dataset is then around 99%, on an event-level resolution.

Even for the individual images in the events, the accuracy is increased due to the conflicting label resolution via maximum event class probability.

In [ ]:
print('Accuracy: {:.3f}%\n'.format(accuracy_score(group_labels_all1, group_preds_all1) * 100))
print(classification_report(group_labels_all1, group_preds_all1, target_names=izw_classes))
label_confusion_matrix(group_labels_all1, group_preds_all1)

In [ ]:
print('Accuracy: {:.3f}%\n'.format(accuracy_score(group_labels_all2, group_preds_all2) * 100))
print(classification_report(group_labels_all2, group_preds_all2, target_names=izw_classes))
label_confusion_matrix(group_labels_all2, group_preds_all2)

---
# Classification analysis - extended events

Exploiting correlations in the 3-image events allowed us to improve the accuracy compared to single-image classification. In the preprocessing phase, we connected 3-image events that occurred in succession (since they likely capture the same extended event).  
We can now try if the longer events let us reduce the error rate further - if an event has a Leopard classification but the events before and after detected a Cheetah, we might be able to correct the Leopard misclassification.

However, we have to be careful with very long events, since it might happen that the presence two different animals really overlapped. (Should check ecological plausibility here.)

For now, we just use the same strategies as for the 3-image events: Classify extended events by the most confident prediction among all images or the sum of all probabilities.

In [ ]:
ext_label_counts, ext_correct_label, ext_event_lens = get_event_labels(val_metadata.groupby('event_key'))
print("Observed label counts: {}".format(ext_label_counts))
print("Correct label in set:  {}".format(ext_correct_label))
print("Num. of events in set: {}".format(ext_event_lens))

---
### Unique-label event accuracy

The unique-label accuracy is increased for extended events compared to 3-image events. This is counter-intuitive, since non-conflicting 3-image events could be still unique in larger events.  
But as the table above shows, some unique 3-events were merged into other events that were already conflicting. Among others, 3 Cheetahs that were misclassified as Leopards were moved.

The accuracy increase is thus mostly an artifact of the data here, the interest should mostly lie in how labeling conflicts in extended events are resolved (see below).

In [ ]:
group_labels, group_preds = unique_label_predictions(val_metadata.groupby('event_key'), normalize_counts=True)
print('Accuracy: {:.3f}%\n'.format(accuracy_score(group_labels, group_preds) * 100))
print(classification_report(group_labels, group_preds, target_names=izw_classes, digits=4))

label_confusion_matrix(group_labels, group_preds)

---
### Conflicting-label event accuracy

Again, classifying an event based on its most confident classification for any image gives the best results - even improved from the 3-image event resolution.

In [ ]:
ext_group_labels_all, ext_group_preds_all, ext_group_labels, ext_group_preds1, ext_group_preds2 = all_label_predictions(val_metadata.groupby('event_key'), normalize_counts=True)
    
print('Accuracy: {:.3f}%\n'.format(accuracy_score(ext_group_labels, ext_group_preds1) * 100))
print(classification_report(ext_group_labels, ext_group_preds1, target_names=izw_classes))

print('Accuracy: {:.3f}%\n'.format(accuracy_score(ext_group_labels, ext_group_preds2) * 100))
print(classification_report(ext_group_labels, ext_group_preds2, target_names=izw_classes))

---
### Overall event accuracy
Overall, using extended events slightly improves upon the 3-event image classification - and both are better than the individual image classification.

HOWEVER: Note that the improvement in accuracy comes mostly from resolving the classification of 3 events (9 images), which confused Leopards and Cheetahs.  
We should not draw overly strong conclusions from such a small sample, but the maximum event class probability mode will be used for event-mislabeling resolution. 


In [ ]:
print('Accuracy: {:.3f}%\n'.format(accuracy_score(ext_group_labels_all, ext_group_preds_all) * 100))
print(classification_report(ext_group_labels_all, ext_group_preds_all, target_names=izw_classes))
label_confusion_matrix(ext_group_labels_all, ext_group_preds_all)